In [1]:
import cv2
from sklearn.cluster import KMeans
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import pandas as pd
import os
from math import floor,sqrt
from moviepy.editor import VideoFileClip
import librosa
from sklearn.preprocessing import StandardScaler, LabelEncoder
import h5py
from sklearn.metrics import precision_score, recall_score, f1_score
import objectDetection as od


## Video Summarization

### Annotations loader

In [2]:
# Load annotations
def load_annotations(anno_file, info_file):
    # Read the annotation and info files
    annotations = pd.read_csv(anno_file, sep='\t', header=None)
    info = pd.read_csv(info_file, sep='\t', header=None)
    
    # Rename columns for better understanding
    annotations.columns = ['video_id', 'category', 'importance_score']
    info.columns = ['category_code', 'video_id', 'title', 'url', 'length']
    
    # print( annotations, info)
    
    return annotations, info

### Preprocessing and Feature Extraction


#### a. Video Processing and Frame Extraction


In [3]:
def extract_frames(video_path, frame_rate=1):
    video = cv2.VideoCapture(video_path)
    count = 0
    success = True
    frames = []
    
    while success:
        success, image = video.read()
        if count % frame_rate == 0 and success:
            frames.append(image)
        count += 1

    video.release()
    return frames

#### b. Audio Extraction


In [4]:

def extract_audio_from_video(video_path, output_audio_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(output_audio_path)
    video.close()

#### c.Audio Features

In [5]:
def extract_audio_features_for_each_frame(audio_path, frame_rate):
    y, sr = librosa.load(audio_path)

    # Calculate the number of audio samples per video frame
    samples_per_frame = sr / frame_rate

    # Initialize an array to store MFCCs for each frame
    mfccs_per_frame = []

    # Iterate over each frame and extract corresponding MFCCs
    for frame in range(int(len(y) / samples_per_frame)):
        start_sample = int(frame * samples_per_frame)
        end_sample = int((frame + 1) * samples_per_frame)

        # Ensure the end sample does not exceed the audio length
        end_sample = min(end_sample, len(y))

        # Extract MFCCs for the current frame's audio segment
        mfccs_current_frame = librosa.feature.mfcc(y=y[start_sample:end_sample], sr=sr, n_mfcc=13)
        mfccs_processed = np.mean(mfccs_current_frame.T, axis=0)
        mfccs_per_frame.append(mfccs_processed)
    print(np.array(mfccs_per_frame).shape)

    return mfccs_per_frame[:len(mfccs_per_frame)-1]


#### d. Feature Extraction (example with visual features using a CNN)


In [6]:
model = VGG16(weights='imagenet', include_top=False)

def extract_visual_features(frames):
    features = []
    for frame in frames:
        if frame is not None:
            img = cv2.resize(frame, (224, 224))  # Resize frame to 224x224
            img = img_to_array(img)        # Convert to array
            img = np.expand_dims(img, axis=0)    # Add batch dimension
            img = preprocess_input(img)          # Preprocess for VGG16
            
            feature = model.predict(img,use_multiprocessing=True,workers=4)
            features.append(feature.flatten())

    return features

### Connect Audio/Annotation/Video

#### Annotation To List

In [7]:
def annotation2List(annotation_features):
    # Make the string '1,1,1,3,2,2,4,4,1' to float list
    annotation_float_array=[]
    for annotation in annotation_features:
        
        if isinstance(annotation,str):
            annotation = annotation.split(',')
        if isinstance(annotation,list):
            for anno in annotation:
                annotation_float_array.append(float(anno))
        else:
            annotation_float_array.append(float(annotation))
    return annotation_float_array

#### Extract Data

In [8]:
import pickle
with open('objects.pkl', 'rb') as f:
    objects = pickle.load(f)

with open('frames.pkl', 'rb') as f:
    frames = pickle.load(f)

with open('labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [9]:
def extractData(video_path, anno_file, info_file):
    # Extract frames from the video
    frames = extract_frames(video_path)
    # print('frames',len(frames)) 

    # # Extract visual features
    visual_features = extract_visual_features(frames) #type: ignore

    # Extract audio
    audio_output_path = 'datasets/extractedAudio/extracted_audio.wav'
    extract_audio_from_video(video_path, audio_output_path) 

    # Extract audio features
    audio_features = extract_audio_features_for_each_frame(audio_output_path,30)

    # # Load annotations
    # annotations, info = load_annotations(anno_file, info_file)
    
    return visual_features, frames,audio_features#type: ignore


#### Kmeans and feature connection

##### One Hot Encoding

In [10]:
def one_hot_encode(objects_in_frame, unique_objects):
    """
    Convert a list of objects detected in a frame to a one-hot encoded vector.
    
    Args:
    objects_in_frame: List of objects detected in a frame.
    unique_objects: List of all unique objects across all frames.

    Returns:
    One-hot encoded vector representing the presence of objects in the frame.
    """
    encoding = [1 if obj in objects_in_frame else 0 for obj in unique_objects]
    return np.array(encoding)


##### Kmeans

In [11]:
def MultiModalKMeans(video_path, anno_file, info_file, num_clusters=None):
    """
    Integrate visual, audio, and annotation features from a video,
    and perform clustering on the combined features.

    :param video_path: Path to the video file.
    :param anno_file: Path to the annotation file.
    :param info_file: Path to the info file.
    :param num_clusters: Number of clusters to use in KMeans.
    :return: Cluster labels for each data point.
    """
    
    # Extract data from video
    visual_features,frames,audio_features=extractData(video_path, anno_file, info_file)
        
    objects = od.detect_objects_in_all_frames(frames,yolo_model,classes) #type: ignore
            
    unique_objects = sorted(list(set([obj for frame_objects in objects for obj in frame_objects]))) #type: ignore

    encoded_objects = [one_hot_encode(frame_objects, unique_objects) for frame_objects in objects] #type: ignore

    # Combine features with padding
    combined_features = []

    for i, frame in enumerate(frames):
        
        # annotation_float_array = annotation2List(annotation_features[i])
        
        combined_feature = np.concatenate([
            np.array(visual_features[i], dtype=float),
            np.array(audio_features[i], dtype=float),
            np.array(encoded_objects[i], dtype=float),
        ])
        combined_features.append(combined_feature)

    # Convert to 2D NumPy array and normalize
    combined_features_array = np.array(combined_features)
    
    combined_features_normalized = StandardScaler().fit_transform(combined_features_array)
         
    print("Number of clusters:", num_clusters)
    
    # Perform clustering
    print("Performing clustering...")
    kmeans = KMeans(n_clusters=num_clusters) #type: ignore
    kmeans.fit(combined_features_normalized)
    
    return [frames, kmeans.labels_]
    # return the frames and the labels


### Summarization

#### Find min cluster

In [12]:
from collections import Counter
def getMinClusster(labels):
    cluster_counts = Counter(labels)

    # Find the cluster with the maximum number of frames
    return min(cluster_counts, key=cluster_counts.get),cluster_counts #type: ignore

#### Frames Selection

In [13]:
def frame_selection(frames, labels):
    """
    Create a summary by selecting frames from the most populous cluster.

    :param frames: List of frames.
    :param labels: Cluster labels for each frame.
    :return: List of frames belonging to the most populous cluster.
    """
    # Count the number of frames in each cluster
    max_cluster,cluster_counts = getMinClusster(labels)

    # Initialize lists for indices and frames
    summary_indices = []  # to measure the importance based on annotation
    summary_frames = []

    # Iterate and select frames and their indices belonging to the most populous cluster
    for index, (frame, label) in enumerate(zip(frames, labels)):
        if label == max_cluster:
            summary_indices.append(index)
            summary_frames.append(frame)

    print(f"Number of frames selected for summary: {len(summary_frames)}, Cluster: {max_cluster}")
    # print all clusters len
    print(f"Cluster counts: {cluster_counts}")
    return summary_frames,summary_indices

def frameSelectionForEachCluster(frames):
    """
    Create a summary by selecting frames from the most populous cluster.

    :param frames: List of frames.
    :param labels: Cluster labels for each frame.
    :return: List of frames belonging to the most populous cluster.
    """
    # Initialize lists for indices and frames
    summary_indices = []  # to measure the importance based on annotation
    summary_frames = []

    # Iterate and select frames and their indices belonging to the most populous cluster
    for index, (frame) in enumerate(frames):
        summary_indices.append(index)
        summary_frames.append(frame)

    print(f"Number of frames selected for summary: {len(summary_frames)}")
    # print all clusters len
    # print(f"Cluster counts: {cluster_counts}")
    return summary_frames,summary_indices

#### Video Creator

In [14]:
def create_video_from_frames(frames, output_path, frame_rate=30):
    if not frames:
        print("No frames to create a video.")
        return 
    # Determine the width and height from the first frame
    height, width, layers = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')#type: ignore
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))#type: ignore

    # Write each frame to the video
    for frame in frames:
        out.write(frame)

    # Release the VideoWriter object
    out.release()

#### Load .Mat file

In [15]:
def decode_titles(encoded_titles, hdf5_file):
    decoded_titles = []
    for ref_array in encoded_titles:
        # Handle the case where each ref_array might contain multiple references
        for ref in ref_array:
            # Dereference each HDF5 object reference to get the actual data
            title_data = hdf5_file[ref]
            # Decode the title
            decoded_title = ''.join(chr(char[0]) for char in title_data)
            decoded_titles.append(decoded_title)
    return decoded_titles


def load_mat_file(file_path,videoID):
    """
    Load a .mat file and return its contents.

    :param file_path: Path to the .mat file.
    :return: Contents of the .mat file.
    """
    with h5py.File(file_path, 'r') as file:
        user_anno_refs=file['tvsum50']['user_anno'][:] # type: ignore
        video_refs=file['tvsum50']['video'][:] # type: ignore

        decoded_videos = decode_titles(video_refs,file)
    
        annotations = []        
        # Get the index from decoded video list to find the annotation for the video
        index = [i for i, x in enumerate(decoded_videos) if x.lower() in videoID.lower()][0]
        
        # Iterate over each reference
        for ref in user_anno_refs: # type: ignore
            # Dereference each HDF5 object reference
            ref_data = file[ref[0]]

            # Convert to NumPy array and add to the annotations list
            annotations.append(np.array(ref_data))
            
        return annotations[index]

#### f1score with ground_truth

In [16]:
def evaluate_frame_selection(ground_truth, summary_indices):
    """
    Evaluate the selected frames by comparing them with the ground truth.
    
    Args:
    ground_truth: Ground truth annotations.
    summary_indices: Indices of the selected frames.
    
    Returns:
    Average importance score, max importance score, and proportion of frames with high importance score.
    """
    
    # Evaluate the selected frames
    selected_importance_scores = ground_truth[summary_indices]
    
    # Calculate metrics
    if selected_importance_scores.size == 0:
        average_importance = 0
        max_importance = 0
        proportion_high_importance = 0
    else:
        average_importance = np.mean(selected_importance_scores)  # Average importance score
        max_importance = np.max(selected_importance_scores)  # Max importance score
        # Calculate the proportion of frames with high importance score
        proportion_high_importance = np.mean(selected_importance_scores >= np.floor(max_importance))
        
    return average_importance, max_importance,proportion_high_importance


In [30]:
def Evaluation(ground_truth_path,summary_indices,videoID):
    
    # Get the ground_truth
    ground_truth = np.array(load_mat_file(ground_truth_path, videoID))
    
    # Find the mean of all annotators
    gold_standard = np.mean(ground_truth, axis=0)
    
    # based on annotators find the avg_importance, max_importance, prop_high_importance
    avg_importance, max_importance, prop_high_importance = evaluate_frame_selection(gold_standard, summary_indices)

    # Thresholding based on max_importance
    threshold=floor(np.mean(avg_importance))
    
    
    # Binary conversion
    binary_ground_truth = np.where(gold_standard >= threshold, 1, 0) 
    
    # Selected frames binary conversion
    selected_frames_binary = np.zeros_like(binary_ground_truth)
    selected_frames_binary[summary_indices] = 1
    
    # Calculate metrics
    precision = precision_score(binary_ground_truth, selected_frames_binary)
    recall = recall_score(binary_ground_truth, selected_frames_binary)  
    f1 = f1_score(binary_ground_truth, selected_frames_binary, average='macro')
    
    # return metrics
    return threshold,precision,recall,f1,avg_importance,max_importance,prop_high_importance


#### KnapSack

In [18]:
def knapsack_for_video_summary(values, weights, capacity, scale_factor=30):
    """
    Apply the 0/1 Knapsack algorithm to select video segments for summarization.

    :param values: List of importance scores for each segment.
    :param weights: List of durations for each segment in seconds.
    :param capacity: Maximum total duration for the summary in seconds.
    :param scale_factor: Factor to scale weights to integers.
    :return: Indices of the segments to include in the summary.
    """
    # Scale weights and capacity
    weights = [int(w * scale_factor) for w in weights]
    capacity = int(capacity * scale_factor)

    n = len(values)
    K = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]

    # Build table K[][] in a bottom-up manner
    for i in range(n + 1):
        for w in range(capacity + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif weights[i-1] <= w:
                K[i][w] = max(values[i-1] + K[i-1][w-weights[i-1]], K[i-1][w])
            else:
                K[i][w] = K[i-1][w]

    # Find the selected segments
    res = K[n][capacity]
    w = capacity
    selected_indices = []

    for i in range(n, 0, -1):
        if res <= 0:
            break
        if res == K[i-1][w]:
            continue
        else:
            selected_indices.append(i-1)
            res = res - values[i-1]
            w = w - weights[i-1]

    selected_indices.reverse()
    return selected_indices


### importance score

In [19]:
def calculate_importance_scores(labels, detected_objects):
    """Here the importance for all the labels together..

    Args:
        labels (_type_): _description_
        detected_objects (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Count the frequency of each label
    label_counts = Counter(labels)

    # Assign a basic importance score based on the inverse frequency of cluster labels
    base_importance_scores = [1 / label_counts[label] for label in labels]

    # Additional importance based on detected objects
    object_importance_scores = []
    for objects in detected_objects:
        if objects:  # if the list is not empty
            # Add extra importance for each detected object
            # You can customize this part based on the type and number of objects
            object_importance_scores.append(len(objects))
        else:
            object_importance_scores.append(0)

    # Combine base importance scores with object importance scores
    # Normalize object importance scores for simplicity
    max_object_score = max(object_importance_scores) if object_importance_scores else 1
    normalized_object_scores = [score / max_object_score for score in object_importance_scores]

    # Final importance score is a combination of base and object scores
    importance_scores = [base + obj for base, obj in zip(base_importance_scores, normalized_object_scores)]
    
    print("Average importance score:", np.mean(importance_scores))
    
    return importance_scores


def calculate_importance_scores_for_cluster(cluster_frames, detected_objects_per_frame):
    """Here the importance for each frame based on objects in frame

    Args:
        cluster_dict (_type_): _description_
        detected_objects_per_frame (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Initialize importance scores for the cluster
    importance_scores = []

    # Calculate object-based importance for each frame in the cluster
    for frame_index in cluster_frames:
        # Calculate object importance
        objects = detected_objects_per_frame[frame_index]
        object_importance = len(objects) if objects else 0

        # Add object importance to the list
        importance_scores.append(object_importance)

    # Normalize the importance scores
    max_importance = max(importance_scores, default=1)
    if max_importance == 0:
        max_importance = 1  # To prevent division by zero

    normalized_importance_scores = [score / max_importance for score in importance_scores]

    return normalized_importance_scores


### Map labels with frames

In [20]:
def map_frames_to_labels_with_indices(frames, labels):
    label_frame_dict = {}
    for label, (frame_index, frame) in zip(labels, enumerate(frames)):
        if label not in label_frame_dict:
            label_frame_dict[label] = []
        label_frame_dict[label].append((frame_index, frame))
    return label_frame_dict


# Code to create summary video

#### Variables

In [21]:
annotation_path='datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
info_path='datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv'

In [22]:
video_path='datasets/ydata-tvsum50-v1_1/video/'
summary_video_path='datasets/summary_videos/'

In [23]:
ground_truth_path='datasets/ydata-tvsum50-v1_1/ground_truth/ydata-tvsum50.mat'

##### Get the list of the videos in the folder

In [24]:
video_list = [video for video in os.listdir(video_path) if video.endswith('.mp4')]  # List comprehension

##### Yolo Model

In [25]:
yolo_model = cv2.dnn.readNetFromDarknet('yolo/yolov3.cfg', 'yolo/yolov3.weights') #type:ignore

with open("yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [26]:
frame_rate = 30  # or whatever your frame rate is
frame_duration = 1 / frame_rate  # Duration of each frame in seconds
capacity = 15  # 15 seconds summary

#### Function for videoSummarizion

In [27]:

def videoSumm(annotation_path=None, info_path=None, video_path=None, summary_video_path=None,video_list=None):
    for video in video_list: #type: ignore
        
        # Kmeans clustering
        # frames,labels = MultiModalKMeans(video_path+video, annotation_path, info_path,num_clusters=11)

        """ USE THIS IN CASE OF CREATING VIDEO FOR EACH CLUSTER"""
        cluster_dict = map_frames_to_labels_with_indices(frames, labels)

        scores = []
        for index, cluster in cluster_dict.items():
            print("Cluster:", index)
            # Extract just the frame indices for importance score calculation
            cluster_frame_indices = [frame_index for frame_index, _ in cluster]

            # Calculate importance scores for this cluster
            importance = calculate_importance_scores_for_cluster(cluster_frame_indices, objects) # Adjust your function as necessary

            # Create a list of durations, one for each frame in the cluster
            durations = [frame_duration] * len(importance)

            # Apply the knapsack algorithm
            summary_indices = knapsack_for_video_summary(importance, durations, capacity)

            # Select Representative Frames (extracting frame part from each tuple)
            summary_frames = [cluster[i][1] for i in summary_indices]  # [1] extracts the frame from (frame_index, frame)

            # Create Summary Video
            create_video_from_frames(summary_frames, f"{summary_video_path}{index}-{video}", 30)

            # Extract original indices for evaluation
            original_indices_for_eval = [cluster[i][0] for i in summary_indices]  # [0] extracts the original frame index

            # Evaluate
            ev = Evaluation(ground_truth_path, original_indices_for_eval, video.split('.')[0])
            scores.append((str(index),) + ev)
            print()
        # Print the table of scores
        from tabulate import tabulate
        scores.sort(key=lambda x: x[3], reverse=True)
        print(tabulate(scores, headers=['videoID','Threshold','Precision', 'Recall', 'F1 Score', 'Avg_importance', 'Max_importance', 'Prop_high_importance'], tablefmt='fancy_grid'))
        return
    
        """
        USE THIS IN CASE OF ONE CLUSTER
        importance=calculate_importance_scores(labels,objects) #type: ignore
                
        # Create a list of durations, one for each frame
        durations = [frame_duration] * len(importance)
        
        # Apply the knapsack algorithm
        summary_indices = knapsack_for_video_summary(importance, durations, capacity)

        # Extract summary frames based on 'summary_indices'
        summary_frames = [frames[i] for i in summary_indices] #type: ignore
        
        create_video_from_frames(summary_frames, summary_video_path+video,30) # Create a video from the selected frames
        
        Evaluation(ground_truth_path,summary_indices,video.split('.')[0])
        break
        """

### Function for object detection

### Code for dynamic annotation and video summarization

In [31]:
videoSumm(annotation_path, info_path, video_path, summary_video_path, video_list)

Cluster: 6

Cluster: 2

Cluster: 9

Cluster: 8

Cluster: 10

Cluster: 3
No frames to create a video.

Cluster: 7

Cluster: 5

Cluster: 1

Cluster: 4

Cluster: 0

╒═══════════╤═════════════╤═════════════╤═══════════╤════════════╤══════════════════╤══════════════════╤════════════════════════╕
│   videoID │   Threshold │   Precision │    Recall │   F1 Score │   Avg_importance │   Max_importance │   Prop_high_importance │
╞═══════════╪═════════════╪═════════════╪═══════════╪════════════╪══════════════════╪══════════════════╪════════════════════════╡
│         8 │           3 │    0.821229 │ 0.7       │  0.87061   │          3.06131 │             3.35 │             0.821229   │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         4 │           2 │    1        │ 0.0971698 │  0.449667  │          2.13625 │             2.4  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼─────

## Results after the video call

In [29]:
╒═══════════╤═════════════╤═════════════╤════════════╤════════════╤══════════════════╤══════════════════╤════════════════════════╕
│   videoID │   Threshold │   Precision │     Recall │  F1 BINARY │   Avg_importance │   Max_importance │   Prop_high_importance │
╞═══════════╪═════════════╪═════════════╪════════════╪════════════╪══════════════════╪══════════════════╪════════════════════════╡
│         8 │           3 │  0.821229   │ 0.7        │ 0.755784   │          3.06131 │             3.35 │             0.821229   │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         2 │           3 │  0.167939   │ 0.104762   │ 0.129032   │          2.61374 │             3    │             0.167939   │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         4 │           2 │  1          │ 0.0971698  │ 0.177128   │          2.13625 │             2.4  │             1          │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         6 │           2 │  0.437778   │ 0.0619497  │ 0.10854    │          1.92333 │             2.4  │             0.437778   │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         9 │           2 │  1          │ 0.0327044  │ 0.0633374  │          2.6024  │             2.7  │             1          │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         0 │           1 │  1          │ 0.0231481  │ 0.0452489  │          1.48125 │             1.5  │             1          │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         5 │           2 │  0.12       │ 0.0169811  │ 0.0297521  │          1.626   │             2    │             0.12       │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         7 │           1 │  1          │ 0.0157697  │ 0.0310497  │          1.5     │             1.65 │             1          │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│        10 │           2 │  0.192771   │ 0.00503145 │ 0.00980693 │          1.93675 │             2.45 │             0.192771   │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         1 │           2 │  0.00888889 │ 0.00125786 │ 0.00220386 │          1.44733 │             2.15 │             0.00888889 │
├───────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         3 │           0 │  0          │ 0          │ 0          │          0       │             0    │             0          │
╘═══════════╧═════════════╧═════════════╧════════════╧════════════╧══════════════════╧══════════════════╧════════════════════════╛

╒═══════════╤═════════════╤═════════════╤═══════════╤═══════════╤══════════════════╤══════════════════╤════════════════════════╕
│   videoID │   Threshold │   Precision │    Recall │ F1 BINARY │   Avg_importance │   Max_importance │   Prop_high_importance │
╞═══════════╪═════════════╪═════════════╪═══════════╪═══════════╪══════════════════╪══════════════════╪════════════════════════╡
│         8 │           3 │    0.821229 │ 0.7       │ 0.755784  │          3.06131 │             3.35 │             0.821229   │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         4 │           2 │    1        │ 0.0971698 │ 0.177128  │          2.13625 │             2.4  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         2 │           2 │    0.984733 │ 0.0811321 │ 0.149913  │          2.61374 │             3    │             0.167939   │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         6 │           1 │    1        │ 0.0651042 │ 0.122249  │          1.92333 │             2.4  │             0.437778   │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         5 │           1 │    1        │ 0.0651042 │ 0.122249  │          1.626   │             2    │             0.12       │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         1 │           1 │    1        │ 0.0651042 │ 0.122249  │          1.44733 │             2.15 │             0.00888889 │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         9 │           2 │    1        │ 0.0327044 │ 0.0633374 │          2.6024  │             2.7  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         0 │           1 │    1        │ 0.0231481 │ 0.0452489 │          1.48125 │             1.5  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         7 │           1 │    1        │ 0.0157697 │ 0.0310497 │          1.5     │             1.65 │             1          │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│        10 │           1 │    1        │ 0.0120081 │ 0.0237312 │          1.93675 │             2.45 │             0.192771   │
├───────────┼─────────────┼─────────────┼───────────┼───────────┼──────────────────┼──────────────────┼────────────────────────┤
│         3 │           0 │    0        │ 0         │ 0         │          0       │             0    │             0          │
╘═══════════╧═════════════╧═════════════╧═══════════╧═══════════╧══════════════════╧══════════════════╧════════════════════════╛

╒═══════════╤═════════════╤═════════════╤═══════════╤════════════╤══════════════════╤══════════════════╤════════════════════════╕
│   videoID │   Threshold │   Precision │    Recall │   F1 MACRO │   Avg_importance │   Max_importance │   Prop_high_importance │
╞═══════════╪═════════════╪═════════════╪═══════════╪════════════╪══════════════════╪══════════════════╪════════════════════════╡
│         8 │           3 │    0.821229 │ 0.7       │  0.87061   │          3.06131 │             3.35 │             0.821229   │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         4 │           2 │    1        │ 0.0971698 │  0.449667  │          2.13625 │             2.4  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         2 │           2 │    0.984733 │ 0.0811321 │  0.434039  │          2.61374 │             3    │             0.167939   │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         6 │           1 │    1        │ 0.0651042 │  0.0611247 │          1.92333 │             2.4  │             0.437778   │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         5 │           1 │    1        │ 0.0651042 │  0.0611247 │          1.626   │             2    │             0.12       │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         1 │           1 │    1        │ 0.0651042 │  0.0611247 │          1.44733 │             2.15 │             0.00888889 │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         9 │           2 │    1        │ 0.0327044 │  0.385748  │          2.6024  │             2.7  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         0 │           1 │    1        │ 0.0231481 │  0.0226244 │          1.48125 │             1.5  │             1          │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         7 │           1 │    1        │ 0.0157697 │  0.0155249 │          1.5     │             1.65 │             1          │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│        10 │           1 │    1        │ 0.0120081 │  0.0118656 │          1.93675 │             2.45 │             0.192771   │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼──────────────────┼──────────────────┼────────────────────────┤
│         3 │           0 │    0        │ 0         │  0         │          0       │             0    │             0          │
╘═══════════╧═════════════╧═════════════╧═══════════╧════════════╧══════════════════╧══════════════════╧════════════════════════╛

SyntaxError: invalid character '╒' (U+2552) (3961589821.py, line 1)